Tool usage

What is Tool :- Functions and services an LLM can utilize to extend its capabilities are named "tools" in Langchain.

Langchain has many tools available.

1. search tools
2. math tools
3. SqL tools

and so on...

When we decide language model to use functions there is actually 2 components to it.

1. Having language model decide which function and what the inputs to that function should be. 
2. Calling that function with those inputs.

Langchain combines these two ideas into something call tool. 

Tool is basically a schema definition for a function which then we can convert to openai function specification and callable.

We have many of these tools built into the package, search tools, math tools, SQL tools.

We will largely focus on creating our own tool.


One thing that we have noticed is that when you create you own chains and agents, a lot of it relies on actually creating your own tools because

what you are trying to do is probably pretty specific to your particular task.

1. Create own tools

2. Build a tool based on an OpenAPI spec

    Predating LLMs, the OpenAPI specification is routinely used by service providers to describe their APIs

3. Select from multiple possible tools - called routing.

In [1]:
import os
from dotenv import load_dotenv

In [2]:
load_dotenv()
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

In [3]:
from langchain.agents import tool # too is a decorator which is used to put on top of function, 

In [5]:
@tool                         # what is does is then automatically convert this function into langchain tool that we can use later.
def search(query:str) -> str:
    """Search for weather online"""
    return "42f"

In [6]:
search.name

'search'

In [7]:
search.description

'Search for weather online'

In [9]:
search.args

{'query': {'title': 'Query', 'type': 'string'}}

all of above will be use while creating openai functions.

we can improve by defining more explicit structure of input schema- important because description of the input is what the language model 

uses to determine what the input should be.

So having a really clear definition for the input becomes important.

We can do this by defining pydantic model, and then when we define the function, we can add arg schema equals search input.

In [12]:
from pydantic import BaseModel, Field

class SearchInput(BaseModel):
    query:str = Field(description="Thing to search for")

In [13]:
@tool(args_schema=SearchInput)
def search(query: str) -> str:
    """Search for the weather online."""
    return "42f"

In [14]:
search.args

{'query': {'description': 'Thing to search for',
  'title': 'Query',
  'type': 'string'}}

In [15]:
search.run("sf")

'42f'

lets create a tool which you give you temperature by entering latitude and longitude

In [42]:
import requests
from pydantic import BaseModel, Field
import datetime


# Define Input schema
class OpenMeteoInput(BaseModel):
    latitude: float = Field(description="Latitude of the location to fetch weather data for ")
    longitude: float = Field(description="Longitude of the location to fetch weather data for")
    
@tool(args_schema= OpenMeteoInput)
def get_current_temperature(latitude: float, longitude: float) -> dict:
    """Fetch current temperature for given coordinates."""
    
    BASE_URL = "https://api.open-meteo.com/v1/forecast?"
    # BASE_URL = "https://api.open-meteo.com/v1/forecast?latitude={latitude}&longitude={longitude}&current_weather=true"
    
    #parameters for the request
    params = {
        'latitude': latitude,
        'longitude': longitude,
        'hourly': 'temperature_2m',
        'forecast_days':1,
    }
    
    # make the request
    response = requests.get(BASE_URL, params=params)
    
    if response.status_code == 200:
        results = response.json()
    else:
        raise Exception(f"API Request failed with status code: {response.status_code}")
    
    current_utc_time = datetime.datetime.utcnow()
    time_list = [datetime.datetime.fromisoformat(time_str.replace('Z', '+00:00')) for time_str in results['hourly']['time']]
    temperature_list = results['hourly']['temperature_2m']
    
    closest_time_index = min(range(len(time_list)), key = lambda i: abs(time_list[i] - current_utc_time))
    current_temperature = temperature_list[closest_time_index]
    
    return f'The current temperature is {current_temperature}C'

In [43]:
get_current_temperature.name

'get_current_temperature'

In [44]:
get_current_temperature.description

'Fetch current temperature for given coordinates.'

In [45]:
get_current_temperature.args

{'latitude': {'description': 'Latitude of the location to fetch weather data for ',
  'title': 'Latitude',
  'type': 'number'},
 'longitude': {'description': 'Longitude of the location to fetch weather data for',
  'title': 'Longitude',
  'type': 'number'}}

what we can also do is convert this tool into exact openAI functions definition and you can do this by doing format_tool_to_openai_function

In [46]:
from langchain.tools.render import format_tool_to_openai_function

In [47]:
format_tool_to_openai_function(get_current_temperature)

{'name': 'get_current_temperature',
 'description': 'Fetch current temperature for given coordinates.',
 'parameters': {'properties': {'latitude': {'description': 'Latitude of the location to fetch weather data for ',
    'type': 'number'},
   'longitude': {'description': 'Longitude of the location to fetch weather data for',
    'type': 'number'}},
  'required': ['latitude', 'longitude'],
  'type': 'object'}}

In [48]:
get_current_temperature({"latitude":13, "longitude":14})

'The current temperature is 33.0C'

In [ ]:
# import requests

# def get_current_temperature(location):
#     latitude = location["latitude"]
#     longitude = location["longitude"]

#     url = (
#         f"https://api.open-meteo.com/v1/forecast?"
#         f"latitude={latitude}&longitude={longitude}&current_weather=true"
#     )

#     response = requests.get(url)
#     if response.status_code != 200:
#         raise Exception(f"API Request failed with status code: {response.status_code}")

#     data = response.json()
#     return data["current_weather"]["temperature"]


In [ ]:
# get_current_temperature({"latitude": 13, "longitude": 14})

33.3